In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
from scipy import signal as sig
from sklearn.decomposition import FastICA
import pywt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import xgboost as xgb
from sklearn.pipeline import Pipeline
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import os
import preprocess_library as pre

In [ ]:
# Import EDF and XML
channels = {
    2: 'EEG(sec)',  # 0-based index for position 3
    3: 'ECG',   # 0-based index for position 4
    4: 'EMG',   # 0-based index for position 5
    5: 'EOG(L)',  # 0-based index for position 6
    6: 'EOG(R)',  # 0-based index for position 7
    7: 'EEG',    # 0-based index for position 8
    10: 'ABDO RES'  # 0-based index for position 11
}

# Create a dict for sampling rate
sampling_rate = {
    2: 125,  # 0-based index for position 3
    3: 125,   # 0-based index for position 4
    4: 125,   # 0-based index for position 5
    5: 125,  # 0-based index for position 6
    6: 125,  # 0-based index for position 7
    7: 125,   # 0-based index for position 8
    10: 125   # 0-based index for position 11
}

raw_data, raw_xml = pre.load_patient_data(1,'Input', channels, 'Output')